In [1]:
# import dependencies
import json
import requests
import time
import pandas as pd
from config import username, pword

In [2]:
# request credentials using username and password
url = "https://api.hooktheory.com/v1/users/auth"
header = {"accept": "application/json", 
          "Content-Type": "application/json",
          "username": username,
          "password": pword}

response = requests.post(url, data=header).json()

# save the key and id from response
key = response['activkey']
user_id = response['id']

# create header used for queries
bearer = f'Bearer {key}'
a_header = {'Authorization':bearer}

In [3]:
c = requests.session()
c.headers = a_header

In [4]:
# set query url and parameters
url = "https://api.hooktheory.com/v1/trends/songs"
params = {'cp':'', 'page':1}

# list progressions we want to check
chords_df = pd.read_csv('Resources/4_chords.csv', header=None)
progressions = chords_df[0].values.tolist()

# initialize a list to store data
songs = []

In [5]:
comm = '''params['cp'] = progressions[0]
r = c.get(url, params = params)
response = r.json()
r.headers['X-Rate-Limit-Reset']'''

In [6]:
# query HookTheory api for information on songs utilizing target chord progressions
# use sleep method to pause script to avoid exceeding rate limits
for cp in progressions:
    params['cp'] = cp
    page = 1
    while page > 0:
        params['page'] = page
        r = c.get(url, params = params)
        count = int(r.headers['X-Rate-Limit-Remaining'])
        if count == 0:
            wait = int(r.headers['X-Rate-Limit-Reset'])
            print(len(songs))
            time.sleep(wait)
        response = r.json()
        if len(response) == 0:
            page = 0
        else:
            for song in response:
                try:
                    song['cp'] = cp
                    songs.append(song)
                except:
                    pass
            page += 1

120
260
380
507
615
719
859
1009
1124
1205
1334
1445
1530
1632
1772
1876
1979
2097
2198
2318
2422
2536
2629
2749
2850
2952
3040
3126
3172
3234
3315
3381
3476
3526
3569
3617
3653
3689
3773
3913
4053
4189
4309
4449
4551
4655
4764
4904
5024
5137
5244
5344
5394
5534
5664
5804
5905
6031
6121
6261
6380
6464
6594
6714
6794
6856
6948
7068
7199
7263
7345
7450
7519
7605
7705
7825
7965
8084
8182
8267
8407
8511
8651
8753
8844
8912
8973
9051
9119
9239
9350
9510
9610
9686
9798
9921
9990
10073
10143
10216
10287
10341
10452
10538
10636
10708
10776
10854
10891
10938
10970
11059
11130
11159
11202
11241
11276
11326
11412
11491
11571
11606
11642
11680
11690
11830
11970
12110
12212
12352
12464
12604
12737
12812
12919
12992
13086
13203
13317
13473
13561
13657
13774
13887
13987
14061
14123
14185
14233
14310
14387
14527
14627
14737
14842
14943
15083
15193
15275
15398
15448
15505
15584
15678
15767
15855
15906
15958


In [7]:
# assign retrieved data to a dataframe
songs_df = pd.DataFrame(songs)

# drop column containing url information
songs_df = songs_df.drop(columns='url')

# display first 30 entries of dataframe
songs_df.head(30)

,artist,cp,section,song
0,3 Doors Down,"1,5,6,4",Intro,Be Like That
1,3 Doors Down,"1,5,6,4",Verse,Be Like That
2,Adele,"1,5,6,4",Chorus,Someone Like You
3,Avril Lavigne,"1,5,6,4",Chorus,Girlfriend
4,Bon Jovi,"1,5,6,4",Chorus,Living On A Prayer
5,Bruno Mars,"1,5,6,4",Chorus,Grenade
6,Carly Rae Jepsen,"1,5,6,4",Chorus,Call Me Maybe
7,Carrie Underwood,"1,5,6,4",Chorus,So Small
8,Carrie Underwood,"1,5,6,4",Verse,So Small
9,Cascada,"1,5,6,4",Verse,Every Time We Touch


In [8]:
# save dataframe as a csv file
songs_df.to_csv('4chord_songs.csv', index=False, header=True)

In [9]:
# display last 30 entries of dataframe
songs_df.tail(30)

,artist,cp,section,song
15971,Eiffel65,"5,2,3,4",Instrumental,Hyperlink - Deep Down
15972,The Queenstons,"5,2,3,4",Verse and Pre-Chorus,Hot One
15973,Avicii vs Nicky Romero,"5,2,3,4",Pre-Chorus and Chorus,I Could Be The One
15974,Flo Rida,"5,2,3,4",Chorus,I Cry
15975,Koji Kondo,"5,2,3,4",Verse,Super Mario 64 - Bowser's Road
15976,John Martin,"5,2,3,4",Chorus,Anywhere For You
15977,Infected Mushroom,"5,2,3,4",Verse,Muse Breaks
15978,Darude,"5,2,3,4",Verse,Feel The Beat
15979,Nhato,"5,2,3,4",Bridge,Miss You
15980,Friendly Fires,"5,2,3,4",Chorus,Kiss Of Life
